## K-Nearest Neighbors Classifier

In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
np.set_printoptions(suppress = True)

import warnings
warnings.simplefilter(action='ignore')


# Importing data sets
from data_import import load_fruits, load_chess, load_music, load_lepiota

fruit_X, fruit_Y = load_fruits()
chess_X, chess_Y = load_chess()
music_X, music_Y = load_music()
lep_X, lep_Y = load_lepiota()


Fruit feature space: (10000, 4)
Fruit label space (10000,)
Chess feature space: (20058, 373)
Chess label space (20058,)
Music feature space: (200, 28)
Music label space (200,)
Lepiota feature space: (8124, 46)
Lepiota label space: (8124,)


### Algorithm

In [2]:
# No training step as KNN memorizes the entire dataset

# Euclidean distance between two feature vectors
def euclidean_distance(a, b):
    return np.sqrt(((a - b) ** 2).sum())
    
# Calculate error and accuracy
def calc_performance(preds, gts):
    e = 0
    n = len(preds)
    e = np.count_nonzero(preds != gts)
    acc = n - e
    return e / n, acc / n

# KNN Algorithm
def knn_classifier(X_train, Y_train, X, y, k):
    y_preds = np.array([])
    
    for sample in X:
           
        dist_label_pairs = []

        for xi, yi in zip(X_train, Y_train):
            dist = euclidean_distance(xi, sample)
            dist_label_pairs.append((dist, yi))

        sorted_dist_label_pairs = sorted(dist_label_pairs, key = lambda x:x[0])

        k_dist_label_pairs = sorted_dist_label_pairs[:k]
        
        k_labels = [pair[1] for pair in k_dist_label_pairs]
        
        # Counting predictions and assigning majority label to sample
        pos_labels = 0 
        neg_labels = 0
        for label in k_labels:
            if label == 1:
                pos_labels += 1
            elif label == -1:
                neg_labels += 1

        # Make the prediction based on counts.
        if pos_labels > neg_labels:
            y_preds = np.append(y_preds, 1)
        else:
            y_preds = np.append(y_preds, -1)
    error, accuracy = calc_performance(y_preds, y)

    return error, accuracy

### Hyper-parameter Tuning (*k*)

In [3]:
# Combines labels (Y) as another column to feature set (X)
def join_data(X, Y):
    return np.hstack((X, Y.reshape(Y.shape[0], 1)))

def cross_validate(data, model, k, kf):
    avg_e = np.array([])
    avg_acc = np.array([])
    for train_index, test_index in kf.split(data):
        error, accuracy = model(data[train_index][:, :data.shape[1] - 1], 
                                data[train_index][:, -1], 
                                data[test_index][:, :data.shape[1] - 1],
                                data[test_index][:, -1],
                                k)
        avg_e = np.append(avg_e, error)
        avg_acc = np.append(avg_acc, accuracy)
    return avg_e.mean(), avg_acc.mean()

In [4]:
def tune_k(X, y, k_list = np.arange(1, 11)):
    lowest_error = 1
    opt_acc = 0
    opt_k = 0
    data = join_data(X, y)
    kf = KFold(n_splits = 5, shuffle = True)

    for k in k_list:
        print('k = {}...'.format(k))
        error, acc = cross_validate(data, knn_classifier, k, kf)
        print('error: {}, acc: {}'.format(error, acc))
        if error < lowest_error:
            opt_k = k
            lowest_error = error
            opt_acc = acc
    print('Best k: {}'.format(opt_k))
    return opt_k, acc